In [1]:
from collections import namedtuple
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','done'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    


In [2]:
import torch
import torchvision
import gym

import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn.functional as F
import gym
from gym.envs.registration import register
register(
   id='FrozenLakeNotSlippery-v0',
   entry_point='gym.envs.toy_text:FrozenLakeEnv',
   kwargs={'map_name' : '4x4', 'is_slippery': False},
   max_episode_steps=100,
   reward_threshold=0.78, # optimum = .8196
)

#env = gym.make('FrozenLake8x8-v0')
#env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLakeNotSlippery-v0')
env.render()


use_cuda = torch.cuda.is_available
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

class q_net(nn.Module):
    def __init__(self):
        super(q_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4)
        

    def forward(self, x):
#         print("Q_Net: Input " + "-" *5)
#         print(x.shape)
#         print(x)
#         print("Q_Net: Input " + "-" *5)
        x = x.view(-1,1)
        x = F.tanh(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = F.tanh(self.linear2(x))
        x = x.view(-1,4)
        #print(x.shape)
        #print(x)
        return x
    
    


SFFF
FHFH
FFFH
HFFG


In [3]:
import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical
import random
random.seed(1999)
import math

# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    #print classname
    #print q_net
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        if not m.bias is None:
            m.bias.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)        
        #m.weight.data.fill_(0.1)
        #if not m.bias is None:
        #    m.bias.data.fill_(0.1)
        print m
        


# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mse = nn.MSELoss(reduce=False)
NUM_EPISODES = 1000000
BATCH_SIZE = 500
GAMMA = 0.9
TARGET_UPDATE = 50
EPS_START = 0.9
EPS_END = 0.0
EPS_DECAY = 1000000
online_net = q_net().to(device)
online_net.apply(weights_init)
target_net = q_net().to(device)
target_net.load_state_dict(online_net.state_dict())
target_net.eval()


memory = ReplayMemory(100000)
#optimizer = optim.RMSprop(online_net.parameters(), lr=0.001)
optimizer = optim.Adam(online_net.parameters(), lr=0.0001)

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    #non_final_mask = torch.tensor(tuple(map(lambda d: d is False,
    #                                      batch.done)), device=device, dtype=torch.uint8).unsqueeze(1)
    # Compute states that are final.
    next_state_final_mask = torch.tensor(tuple(map(lambda d: d in [5,7,11,12],
                                          batch.next_state)), device=device, dtype=torch.uint8).unsqueeze(1) 
    next_state_finak_list = [d for d in batch.next_state if d in [5,7,11,12] ]
    
    
    #non_final_next_states = torch.cat([FloatTensor([s]) for s,d in zip(batch.next_state,batch.done)
    #                                            if d is False])
    
    #state_batch = torch.cat([torch.FloatTensor([s]) for s in batch.state])
    state_batch = FloatTensor(batch.state)
    state_batch = state_batch.view(BATCH_SIZE, 1)
    next_state_batch = FloatTensor(batch.next_state)
    next_state_batch = next_state_batch.view(BATCH_SIZE, 1)
    #action_batch = torch.cat([torch.LongTensor([[a.item()]]) for a in batch.action])
    action_batch = LongTensor(batch.action).view(BATCH_SIZE,1)
    #reward_batch = torch.cat([torch.tensor([r]) for r in batch.reward])
    reward_batch = Tensor(batch.reward).view(BATCH_SIZE,1)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
#     print("state_batch "+"-" * 10)
#     print(state_batch.shape)
#     print("action_batch "+"-" * 10)
#     print(action_batch.shape)
    state_action_values = online_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # next_state_values = torch.zeros(BATCH_SIZE, device=device).view(BATCH_SIZE,1)
#     print("non_final_mask")
#     print(non_final_mask.shape)
    #next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # DDQN ------------------------
    # Y_ddqn = R_next + GAMMA * Q_target(S_next, argmax_a Q_online(S_next, a) )
    # Below is the argmax_a Q_online(S_next,a)
    argmax_a_online_net_next_state = online_net(next_state_batch).max(1)[1].detach().view(BATCH_SIZE,1)
    
    # Below is the actual Q_target
    next_state_values = target_net(next_state_batch).detach().gather(1, argmax_a_online_net_next_state)
    # DDQN ------------------------
#     print "next_state_values"
#     print next_state_values.shape
#     print next_state_values.type()
#     print "final_mask"
#     print final_mask.shape
#     print  final_mask.type()
#     print "next_state_values[final_mask]"
#     print next_state_values[final_mask].shape
#     print next_state_values[final_mask].type()
    next_state_values[next_state_final_mask] = torch.zeros(len(next_state_finak_list), device=device).view(len(next_state_finak_list))
#     print("next_state_values")
#     print(next_state_values.shape)
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
#     print("expected_state_action_values")
#     print(expected_state_action_values.shape)

    # Compute Huber loss (this is like MSE , but less sensitive to outliers )
    # loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    #loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
    loss = mse( state_action_values, expected_state_action_values)
    debug = False
    if debug:
        print("-" * 40)
        print("States")
        print(state_batch)
        print("Target Q")
        print(expected_state_action_values)
        print(expected_state_action_values.shape)
        print("Actual Q")
        print(state_action_values)
        print(state_action_values.shape)
        print("Loss")
        print(loss)
        print(loss.shape)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward(torch.ones(len(state_action_values), device=device).unsqueeze(1))
    
    # param in online_net.parameters():
    #    param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    if debug: 
        print("AFTER OPTIMIZATION:")
        print("New actual Q")
        print(online_net(state_batch).gather(1, action_batch))
    
    

Linear(in_features=1, out_features=20, bias=True)
Linear(in_features=20, out_features=4, bias=True)


In [6]:
score = []
times_trained = 0
times_reach_goal = 0
steps_done = 0


# TO BE DELETED_______
#steps_done = 40000000
#EPS_END = 0.0

# TO BE DELETED_______


for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action
    episode_series = []
    reward = 0
    episode_step = 0
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = FloatTensor(np_observation)
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        q_values = online_net(observation_tensor)
        if sample >= eps_threshold: 
            #print "observation_tensor"
            #print observation_tensor.type()
            
            action = q_values.max(1)[1] # First 1 is the dimension, second 1 is the index (this is argmax)
        else:
            action = torch.LongTensor([[random.randint(0,3)]], device=device)
        
            
        #break
        # Execute action in environment.
        old_state = observation                    
            
        
        observation, reward, done, info = env.step(action.item()) 
        new_state = observation
        
        

        
        
#         if done and reward != 1.0:
#             if episode_step > 50:
#                 reward = -0.2
#             if episode_step > 80:
#                 reward = -0.5
#             if new_state in [5,7,11,12]:
#                 reward = -1.0

        
        
        # Store the transition in memory
        #if old_state != new_state:

        memory.push(old_state, action, new_state, reward, done)
        if k%5000 == 0:
            #print("old_state != new_state")
            #print(old_state != new_state)
            #print("oldstate " + str(old_state) + " newstate " + str(new_state))
            print("q_values ")
            print(q_values)
            print("On state="+ str(old_state) + ", selected action=" + str(action.item()) + " , " + \
              "Random? " + str( sample < eps_threshold ))
            print("new state="+ str(new_state) + ", done="+str(done) + \
             ". Reward: " + str(reward))

        # Perform one step of the optimization (on the target network)
        if k > BATCH_SIZE :
            optimize_model()
            times_trained = times_trained + 1

        episode_step += 1
        #env.render()
    if k % TARGET_UPDATE == 0:
        target_net.load_state_dict(online_net.state_dict())
    
    if k % 1000 ==0:
        print_q_table()
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: \
              {}. Times reached goal: {}. \
              Steps done: {}. EPS_DECAY: {}. EPS_THRESHOLD: {}.".format(k, len(episode_series), \
                                                                    reward, np.mean(score), times_trained, \
                                                                       times_reach_goal, steps_done, \
                                                                       EPS_DECAY, eps_threshold))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        

    
    
    
    if reward > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    

q_values 
tensor(1.00000e-02 *
       [[-1.6591, -3.7930,  0.8807,  2.0332]], device='cuda:0')
On state=0, selected action=0 , Random? True
new state=0, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[-1.6591, -3.7930,  0.8807,  2.0332]], device='cuda:0')
On state=0, selected action=2 , Random? True
new state=1, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[-1.6737, -3.6644,  1.0493,  2.1236]], device='cuda:0')
On state=1, selected action=2 , Random? True
new state=2, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[-1.6884, -3.5371,  1.2173,  2.2136]], device='cuda:0')
On state=2, selected action=3 , Random? True
new state=2, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[-1.6884, -3.5371,  1.2173,  2.2136]], device='cuda:0')
On state=2, selected action=2 , Random? True
new state=3, done=False. Reward: 0.0
q_values 
tensor(1.00000e-02 *
       [[-1.7029, -3.4120,  1.3843,  2.3026]], device='cuda:0')
On state=3, selected

 state (0)  A[0]:(0.0870576873422) A[1]:(0.102145478129) A[2]:(0.0535888075829) A[3]:(0.0885259285569)
 state (1)  A[0]:(0.0964293852448) A[1]:(0.111839033663) A[2]:(0.0747825056314) A[3]:(0.0925482362509)
 state (2)  A[0]:(0.105774983764) A[1]:(0.121488578618) A[2]:(0.0959315821528) A[3]:(0.096560664475)
 state (3)  A[0]:(0.115090340376) A[1]:(0.131091117859) A[2]:(0.117007471621) A[3]:(0.100562304258)
 state (4)  A[0]:(0.124371387064) A[1]:(0.140643745661) A[2]:(0.137981981039) A[3]:(0.104552291334)
 state (5)  A[0]:(0.133614107966) A[1]:(0.150143578649) A[2]:(0.158827349544) A[3]:(0.108529746532)
 state (6)  A[0]:(0.142814561725) A[1]:(0.159587994218) A[2]:(0.179516538978) A[3]:(0.11249383539)
 state (7)  A[0]:(0.151968851686) A[1]:(0.168974280357) A[2]:(0.200023293495) A[3]:(0.116443708539)
 state (8)  A[0]:(0.161073192954) A[1]:(0.178299933672) A[2]:(0.220322296023) A[3]:(0.120378553867)
 state (9)  A[0]:(0.17012386024) A[1]:(0.187562480569) A[2]:(0.240389481187) A[3]:(0.124297559

 state (0)  A[0]:(0.0895830467343) A[1]:(0.101614668965) A[2]:(0.0496598929167) A[3]:(0.0883281305432)
 state (1)  A[0]:(0.0970922186971) A[1]:(0.109980262816) A[2]:(0.0693174079061) A[3]:(0.0910073444247)
 state (2)  A[0]:(0.104633189738) A[1]:(0.118351064622) A[2]:(0.0891004055738) A[3]:(0.0937178358436)
 state (3)  A[0]:(0.112200655043) A[1]:(0.126723304391) A[2]:(0.108975149691) A[3]:(0.0964566096663)
 state (4)  A[0]:(0.119789130986) A[1]:(0.135093182325) A[2]:(0.128906697035) A[3]:(0.0992204770446)
 state (5)  A[0]:(0.12739290297) A[1]:(0.143456757069) A[2]:(0.148859098554) A[3]:(0.102006115019)
 state (6)  A[0]:(0.135006144643) A[1]:(0.151810184121) A[2]:(0.168795734644) A[3]:(0.104809977114)
 state (7)  A[0]:(0.142622888088) A[1]:(0.160149440169) A[2]:(0.188679784536) A[3]:(0.107628449798)
 state (8)  A[0]:(0.150237083435) A[1]:(0.168470561504) A[2]:(0.208474367857) A[3]:(0.110457785428)
 state (9)  A[0]:(0.157842606306) A[1]:(0.176769584417) A[2]:(0.228143021464) A[3]:(0.11329

 state (0)  A[0]:(0.174612358212) A[1]:(0.186580345035) A[2]:(0.195264488459) A[3]:(0.184555619955)
 state (1)  A[0]:(0.186041280627) A[1]:(0.203898206353) A[2]:(0.16324698925) A[3]:(0.181176200509)
 state (2)  A[0]:(0.195140883327) A[1]:(0.217340022326) A[2]:(0.10784560442) A[3]:(0.17414213717)
 state (3)  A[0]:(0.20440505445) A[1]:(0.231121495366) A[2]:(0.0558212064207) A[3]:(0.166832655668)
 state (4)  A[0]:(0.216274023056) A[1]:(0.249433472753) A[2]:(0.0352969542146) A[3]:(0.163020163774)
 state (5)  A[0]:(0.231797680259) A[1]:(0.274264872074) A[2]:(0.060232359916) A[3]:(0.165307968855)
 state (6)  A[0]:(0.25061866641) A[1]:(0.30517911911) A[2]:(0.128676190972) A[3]:(0.174425572157)
 state (7)  A[0]:(0.271748840809) A[1]:(0.340451687574) A[2]:(0.229185089469) A[3]:(0.189611285925)
 state (8)  A[0]:(0.294188082218) A[1]:(0.37814244628) A[2]:(0.346678346395) A[3]:(0.209411203861)
 state (9)  A[0]:(0.317155390978) A[1]:(0.416610211134) A[2]:(0.466533333063) A[3]:(0.232298463583)
 stat

 state (0)  A[0]:(0.207345858216) A[1]:(0.231802731752) A[2]:(0.184103310108) A[3]:(0.208010151982)
 state (1)  A[0]:(0.202408269048) A[1]:(0.0152368303388) A[2]:(0.210252985358) A[3]:(0.175735160708)
 state (2)  A[0]:(0.204491361976) A[1]:(0.117405869067) A[2]:(0.165985360742) A[3]:(0.205773696303)
 state (3)  A[0]:(0.204622685909) A[1]:(0.219640597701) A[2]:(0.0796986967325) A[3]:(0.221006885171)
 state (4)  A[0]:(0.202881395817) A[1]:(0.256819665432) A[2]:(0.0298941750079) A[3]:(0.209459617734)
 state (5)  A[0]:(0.202706441283) A[1]:(0.246373921633) A[2]:(0.0365951023996) A[3]:(0.180946499109)
 state (6)  A[0]:(0.208564311266) A[1]:(0.232739269733) A[2]:(0.0904122889042) A[3]:(0.154737278819)
 state (7)  A[0]:(0.223021402955) A[1]:(0.246266916394) A[2]:(0.185129448771) A[3]:(0.145420446992)
 state (8)  A[0]:(0.245862036943) A[1]:(0.294919937849) A[2]:(0.311091274023) A[3]:(0.157499194145)
 state (9)  A[0]:(0.275074362755) A[1]:(0.37118768692) A[2]:(0.451021671295) A[3]:(0.1879984587

...

 state (0)  A[0]:(0.532789945602) A[1]:(0.590682506561) A[2]:(0.590478658676) A[3]:(0.531740188599)
 state (1)  A[0]:(0.533221006393) A[1]:(-0.000174306333065) A[2]:(0.656062960625) A[3]:(0.590502262115)
 state (2)  A[0]:(0.592284679413) A[1]:(0.728952646255) A[2]:(0.591003596783) A[3]:(0.656099081039)
 state (3)  A[0]:(0.657832205296) A[1]:(-0.219640344381) A[2]:(0.540426790714) A[3]:(0.517800986767)
 state (4)  A[0]:(0.592524945736) A[1]:(0.655930995941) A[2]:(0.000232577323914) A[3]:(0.53265106678)
 state (5)  A[0]:(0.167211934924) A[1]:(0.928622722626) A[2]:(-0.195260390639) A[3]:(0.528429508209)
 state (6)  A[0]:(0.00409654481336) A[1]:(0.809909999371) A[2]:(0.000140190124512) A[3]:(0.656988322735)
 state (7)  A[0]:(0.626845359802) A[1]:(-0.248132169247) A[2]:(0.309757322073) A[3]:(0.881797134876)
 state (8)  A[0]:(0.658214986324) A[1]:(-5.3271651268e-06) A[2]:(0.728896021843) A[3]:(0.591342449188)
 state (9)  A[0]:(0.657523274422) A[1]:(0.80997800827) A[2]:(0.809928715229) A[3]:(

 state (0)  A[0]:(0.531601786613) A[1]:(0.590443134308) A[2]:(0.590408086777) A[3]:(0.528941273689)
 state (1)  A[0]:(0.531187653542) A[1]:(0.00046998259495) A[2]:(0.656045436859) A[3]:(0.587685585022)
 state (2)  A[0]:(0.590318799019) A[1]:(0.729059815407) A[2]:(0.590089440346) A[3]:(0.653233647346)
 state (3)  A[0]:(0.656860589981) A[1]:(-0.216925159097) A[2]:(0.539077341557) A[3]:(0.512693226337)
 state (4)  A[0]:(0.591693997383) A[1]:(0.656167387962) A[2]:(-0.00133526243735) A[3]:(0.526328921318)
 state (5)  A[0]:(0.165799468756) A[1]:(0.928652584553) A[2]:(-0.196968510747) A[3]:(0.521318674088)
 state (6)  A[0]:(0.00169956521131) A[1]:(0.810080468655) A[2]:(-0.00189912086353) A[3]:(0.65063893795)
 state (7)  A[0]:(0.624581098557) A[1]:(-0.247704580426) A[2]:(0.308289170265) A[3]:(0.87886351347)
 state (8)  A[0]:(0.655260443687) A[1]:(4.73856925964e-05) A[2]:(0.72910040617) A[3]:(0.581343531609)
 state (9)  A[0]:(0.654440522194) A[1]:(0.810010135174) A[2]:(0.810040712357) A[3]:(-0.

 state (0)  A[0]:(0.531928062439) A[1]:(0.590600013733) A[2]:(0.590536952019) A[3]:(0.530130505562)
 state (1)  A[0]:(0.531933248043) A[1]:(3.51816415787e-05) A[2]:(0.656167387962) A[3]:(0.589769601822)
 state (2)  A[0]:(0.590911209583) A[1]:(0.729074954987) A[2]:(0.590722799301) A[3]:(0.655700743198)
 state (3)  A[0]:(0.656171262264) A[1]:(-0.217723205686) A[2]:(0.540204405785) A[3]:(0.517480790615)
 state (4)  A[0]:(0.590438961983) A[1]:(0.656207501888) A[2]:(0.000309109687805) A[3]:(0.532142043114)
 state (5)  A[0]:(0.164301767945) A[1]:(0.928704559803) A[2]:(-0.195272311568) A[3]:(0.527749657631)
 state (6)  A[0]:(0.00120645703282) A[1]:(0.810073077679) A[2]:(0.000144720077515) A[3]:(0.656175911427)
 state (7)  A[0]:(0.625096559525) A[1]:(-0.247940957546) A[2]:(0.309858202934) A[3]:(0.881325244904)
 state (8)  A[0]:(0.65663677454) A[1]:(0.000203147530556) A[2]:(0.729167580605) A[3]:(0.590406179428)
 state (9)  A[0]:(0.65631711483) A[1]:(0.810063719749) A[2]:(0.810126662254) A[3]:(0

 state (0)  A[0]:(0.53119122982) A[1]:(0.590483188629) A[2]:(0.590462386608) A[3]:(0.531013906002)
 state (1)  A[0]:(0.531282782555) A[1]:(-3.38032841682e-05) A[2]:(0.656093835831) A[3]:(0.590262770653)
 state (2)  A[0]:(0.590485692024) A[1]:(0.729023694992) A[2]:(0.590473771095) A[3]:(0.656036376953)
 state (3)  A[0]:(0.656268179417) A[1]:(-0.218322843313) A[2]:(0.539755284786) A[3]:(0.517144203186)
 state (4)  A[0]:(0.590448081493) A[1]:(0.656111299992) A[2]:(-0.00025475025177) A[3]:(0.531480193138)
 state (5)  A[0]:(0.163611128926) A[1]:(0.928619205952) A[2]:(-0.19541208446) A[3]:(0.526981294155)
 state (6)  A[0]:(-0.000303566455841) A[1]:(0.810006976128) A[2]:(5.66244125366e-05) A[3]:(0.655815660954)
 state (7)  A[0]:(0.624260008335) A[1]:(-0.247785151005) A[2]:(0.309722840786) A[3]:(0.881498098373)
 state (8)  A[0]:(0.656375169754) A[1]:(-1.83060765266e-05) A[2]:(0.728947997093) A[3]:(0.590626895428)
 state (9)  A[0]:(0.656374335289) A[1]:(0.809990286827) A[2]:(0.809962689877) A[3

 state (0)  A[0]:(0.532827734947) A[1]:(0.590501189232) A[2]:(0.590471088886) A[3]:(0.531642198563)
 state (1)  A[0]:(0.53241455555) A[1]:(0.000198662281036) A[2]:(0.6560972929) A[3]:(0.590085268021)
 state (2)  A[0]:(0.591420412064) A[1]:(0.728976547718) A[2]:(0.590568244457) A[3]:(0.655997395515)
 state (3)  A[0]:(0.657168030739) A[1]:(-0.218171328306) A[2]:(0.53993088007) A[3]:(0.519045472145)
 state (4)  A[0]:(0.591712117195) A[1]:(0.655946612358) A[2]:(-0.000321984291077) A[3]:(0.534364104271)
 state (5)  A[0]:(0.16584572196) A[1]:(0.928639888763) A[2]:(-0.196107491851) A[3]:(0.530310571194)
 state (6)  A[0]:(0.00261014117859) A[1]:(0.809897780418) A[2]:(-0.000926136679482) A[3]:(0.658368945122)
 state (7)  A[0]:(0.625910520554) A[1]:(-0.248257100582) A[2]:(0.308763355017) A[3]:(0.882060527802)
 state (8)  A[0]:(0.65710747242) A[1]:(-0.000345528125763) A[2]:(0.728600144386) A[3]:(0.59000146389)
 state (9)  A[0]:(0.656688213348) A[1]:(0.809836506844) A[2]:(0.809769392014) A[3]:(-0.

In [5]:
def print_q_table():
    for i in range(16):
        st = np.array(i)
        st = np.expand_dims(st, axis=0)
        q_vals = online_net(FloatTensor(st))
        outp = " state (" +str(i) + ") "
        n = 0
        for tensr in q_vals:
            for cell in tensr:
                outp = outp + " A[" + str(n) + "]:(" + str(cell.item()) + ")"
                n += 1
        print(outp)
        
print_q_table()
print_q_table()

 state (0)  A[0]:(-0.016590623185) A[1]:(-0.0379296652973) A[2]:(0.00880737882107) A[3]:(0.0203321184963)
 state (1)  A[0]:(-0.0167372319847) A[1]:(-0.0366442278028) A[2]:(0.0104934358969) A[3]:(0.0212363991886)
 state (2)  A[0]:(-0.0168837141246) A[1]:(-0.0353712290525) A[2]:(0.0121734226122) A[3]:(0.022135829553)
 state (3)  A[0]:(-0.0170289520174) A[1]:(-0.0341202318668) A[2]:(0.0138428043574) A[3]:(0.0230262875557)
 state (4)  A[0]:(-0.0171718932688) A[1]:(-0.0329002924263) A[2]:(0.0154972476885) A[3]:(0.0239038150758)
 state (5)  A[0]:(-0.0173115506768) A[1]:(-0.0317197963595) A[2]:(0.0171326845884) A[3]:(0.0247646775097)
 state (6)  A[0]:(-0.0174470487982) A[1]:(-0.0305862799287) A[2]:(0.018745386973) A[3]:(0.0256054457277)
 state (7)  A[0]:(-0.0175776146352) A[1]:(-0.0295063517988) A[2]:(0.0203320086002) A[3]:(0.0264229979366)
 state (8)  A[0]:(-0.0177025925368) A[1]:(-0.0284855756909) A[2]:(0.0218896120787) A[3]:(0.0272145923227)
 state (9)  A[0]:(-0.0178214609623) A[1]:(-0.027